In [395]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)

In [396]:
def unzip(path_to_zip_file, directory_to_extract_to):
    """
    Args:
        path_to_zip_file (str): 압축파일 경로
        directory_to_extract_to (str): 압축 풀고 저장할 경로
    """
    import zipfile
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)


In [397]:
data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)
data = data.drop(["Unnamed: 0", "http.request.uri.query", "http.file_data", "http.request.full_uri", 
                  "tcp.options", "http.request.uri.path", "http.request.uri.query.dict", "http.file_data.dict", "http.request.uri.scheme"], axis="columns")
data["http.data"] = data["http.data"].fillna("")
original_data = data.copy()

# RNN학습시, URL정보를 바탕으로 분류를 진행할 수 있으므로 URL 정보를 삭제해야함.
data["tcp.payload"] = original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([_ for _ in x.split(" ") if not (_.startswith("/DVWA") and _.endswith(".php"))]))

C:\Users\ParkJaehyung\AppData\Local\Temp\ipykernel_30516\524550114.py:1: DtypeWarning: Columns (3,6,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)


In [398]:
# Tokenize values
tokenizer = Tokenizer(oov_token="<NUMBER>")
tokenizer.fit_on_texts(original_data["http.data"])
# word_index = tokenizer.word_index
# word_index = {key: val for key, val in word_index.items() if not re.fullmatch("\d+",key)} # erase number tokenized.
# tokenizer.word_index = {key: i for i, key in enumerate(word_index.keys())} # re index
padded = pad_sequences(tokenizer.texts_to_sequences(original_data["http.data"]), padding="post", value=0)
data["http.data"] = padded.tolist()

In [400]:
# Tokenize tcp.payload
tokenizer_payload = Tokenizer(oov_token="<PAD>")
tokenizer_payload.fit_on_texts(original_data["tcp.payload"].astype("str"))
padded = pad_sequences(tokenizer_payload.texts_to_sequences(original_data["tcp.payload"].astype("str")), padding="post", value=0)
data["tcp.payload"] = padded.tolist()

In [401]:
data.head()

,ip.src_host,frame.time,tcp.checksum,http.response,arp.src.proto_ipv4,http.content_length,Attack_type,arp.dst.proto_ipv4,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,http.referer,ip.dst_host,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.payload,tcp.srcport,arp.opcode,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
29070,192.168.0.170,2021 18:26:24.439558000,0x00006d37,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,1.639718e+09,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",51530.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
79952,192.168.0.170,2021 18:06:03.716141000,0x00003099,0.0,0,0.0,XSS,0,0.0,0.0,0.0,119.0,0.0,0,192.168.0.128,1.0,3.781170e+09,1.0,80.0,0x00000011,485.0,0.0,"[36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",59370.0,0.0,True,False,False,False,False,False,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11546,192.168.0.170,2021 17:17:54.082790000,0x0000d010,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,5.527170e+08,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",58294.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
89586,192.168.0.170,2021 19:24:56.604410000,0x0000bcb2,0.0,0,0.0,XSS,0,0.0,0.0,0.0,103.0,0.0,0,192.168.0.128,1.0,3.692715e+09,1.0,80.0,0x00000011,442.0,0.0,"[36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",60778.0,0.0,True,False,False,False,False,False,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
41380,192.168.0.170,2021 19:07:44.535206000,0x0000d303,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,1.196470e+08,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",58188.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [402]:
original_data.head()

,ip.src_host,frame.time,tcp.checksum,http.response,arp.src.proto_ipv4,http.content_length,Attack_type,arp.dst.proto_ipv4,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,http.referer,ip.dst_host,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.payload,tcp.srcport,arp.opcode,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
29070,192.168.0.170,2021 18:26:24.439558000,0x00006d37,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,1.639718e+09,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,51530.0,0.0,False,True,False,False,True,False,192.168.0.128,
79952,192.168.0.170,2021 18:06:03.716141000,0x00003099,0.0,0,0.0,XSS,0,0.0,0.0,0.0,119.0,0.0,0,192.168.0.128,1.0,3.781170e+09,1.0,80.0,0x00000011,485.0,0.0,NaN,59370.0,0.0,True,False,False,False,False,False,NaN,
11546,192.168.0.170,2021 17:17:54.082790000,0x0000d010,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,5.527170e+08,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,58294.0,0.0,False,True,False,False,True,False,192.168.0.128,
89586,192.168.0.170,2021 19:24:56.604410000,0x0000bcb2,0.0,0,0.0,XSS,0,0.0,0.0,0.0,103.0,0.0,0,192.168.0.128,1.0,3.692715e+09,1.0,80.0,0x00000011,442.0,0.0,NaN,60778.0,0.0,True,False,False,False,False,False,NaN,
41380,192.168.0.170,2021 19:07:44.535206000,0x0000d303,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,1.196470e+08,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,58188.0,0.0,False,True,False,False,True,False,192.168.0.128,


In [403]:
def zipFile(fileName):
    import shutil
    shutil.make_archive(fileName,"zip", fileName)

In [404]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3)

In [405]:
labels = {key: i for i, key in enumerate(test["Attack_type"].unique())}
def to_onehot(x):
    l = [0 for _ in range(labels.__len__())]
    l[labels[x]] = 1
    return l

In [406]:
payload_test_X ,RNN_test_X, test_X, test_Y =  np.stack(test["tcp.payload"]),np.stack(test["http.data"]), test.drop(["Attack_type", "http.data"], axis="columns"), np.stack(test["Attack_type"].apply(to_onehot))
payload_train_X ,RNN_train_X, train_X, train_Y =  np.stack(train["tcp.payload"]),np.stack(train["http.data"]), train.drop(["Attack_type", "http.data"], axis="columns"), np.stack(train["Attack_type"].apply(to_onehot))

In [407]:
vocab_size = tokenizer.word_index.keys().__len__()
input_dim = RNN_train_X.shape[1]

In [409]:
# layers=[
# keras.layers.Embedding(1000, 128, input_length=1000),
# keras.layers.LSTM(64, activation="tanh", dropout=0.5, return_sequences=True),
# # keras.layers.Dense(100, activation="relu"),
# # keras.layers.Dense(100, activation="relu"),
# keras.layers.Dense(3, activation="softmax")
# ]
import os
RNN_MODEL_PATH:str = "logs/http_data_RNN.model"
if "http_data_RNN.model.zip" in os.listdir(os.getcwd()) or RNN_MODEL_PATH:
    unzip(f"{RNN_MODEL_PATH}.zip", f"{RNN_MODEL_PATH}")
    RNN_model = keras.models.load_model(f"{RNN_MODEL_PATH}")
else:
    RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(vocab_size, 128, input_length=input_dim),
            keras.layers.LSTM(128, dropout=0.7, recurrent_dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
        optimizer=tf.keras.optimizers.Adam(learning_rate=.005), 
        metrics=["accuracy"],
    )
    RNN_model.fit(RNN_train_X, train_Y, epochs=30, batch_size=128)
    RNN_model.save("http_data_RNN.model")
    zipFile("http_data_RNN.model")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 36: invalid continuation byte

In [410]:
PAYLOAD_RNN_MODEL_PATH:str = "logs/tcp_payload_RNN.model"
if "tcp_payload_RNN.model.zip" in os.listdir(os.getcwd()) or PAYLOAD_RNN_MODEL_PATH:
    unzip(f"{PAYLOAD_RNN_MODEL_PATH}.zip", f"{PAYLOAD_RNN_MODEL_PATH}")
    payload_RNN_model = keras.models.load_model(f"{PAYLOAD_RNN_MODEL_PATH}")
else:
    payload_RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(vocab_size, 128, input_length=input_dim),
            keras.layers.LSTM(128, dropout=0.7, recurrent_dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    payload_RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
        optimizer=tf.keras.optimizers.Adam(learning_rate=.005), 
        metrics=["accuracy"],
    )
    payload_RNN_model.fit(payload_train_X, train_Y, epochs=5, batch_size=128)
    payload_RNN_model.save("tcp_payload_RNN.model")
    zipFile("tcp_payload_RNN.model")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 36: invalid continuation byte

In [384]:
RNN_model.predict(RNN_test_X)

875/875 [==============================] - 45s 49ms/step


array([[0.80760205, 0.14172995, 0.03181341, 0.01885448],
       [0.80760205, 0.14172995, 0.03181341, 0.01885448],
       [0.80760205, 0.14172995, 0.03181341, 0.01885448],
       ...,
       [0.80760205, 0.14172995, 0.03181341, 0.01885448],
       [0.80760205, 0.14172995, 0.03181341, 0.01885448],
       [0.80760205, 0.14172995, 0.03181341, 0.01885448]], dtype=float32)

In [386]:
test_Y

61745    0
79468    1
16999    0
3337     0
53065    0
        ..
57696    0
43319    0
65847    0
62927    0
22046    0
Name: Attack_type, Length: 27971, dtype: int64

In [ ]:
payload_prediction = payload_RNN_model.predict(payload_test_X)


In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [ ]:
precision_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")


In [ ]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")


In [ ]:
confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1))
